Here is the code to release the queries with the parameters you have selected using the OpenDP library

In [ ]:
# !pip install opendp

In [ ]:
# notebook imports
import opendp.prelude as dp
import math
import pandas as pd
import numpy as np
dp.enable_features("contrib")
dp.enable_features("honest-but-curious")
from typing import List

## Query: age_histogram

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv')

In [ ]:
query_info = {"export_selected_query": "age_histogram", "export_mechanism": "laplace", "export_epsilon": 0.25, "url": "https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv", "export_data_type": "continuous", "export_lower_bound": 18, "export_upper_bound": 93, "export_num_bins": 10}

In [ ]:
def at_delta(meas, delta):
    meas = dp.c.make_zCDP_to_approxDP(meas)
    return dp.c.make_fix_delta(meas, delta)

In [ ]:
# Load data and get parameters
df = pd.read_csv('https://raw.githubusercontent.com/lpanavas/DPEducationDatasets/master/PUMS_california_demographics_1000.csv')
query_type = 'age_histogram'
mechanism = 'laplace'
epsilon = 0.25
column_name = 'age'
column_type = df[column_name].dtype
min_val = 18
max_val = 93
num_bins = 10
bin_edges = np.linspace(min_val, max_val, num_bins + 1, dtype=column_type)

In [ ]:
binner = dp.t.make_find_bin(dp.vector_domain(dp.atom_domain(T=int)), dp.symmetric_distance(), edges=bin_edges[1:-1])
histogram = (
    dp.t.make_split_dataframe(separator=",", col_names=list(df.columns)) >>
    dp.t.make_select_column(column_name, str) >>
    dp.t.then_cast_default(int) >> 
    binner>>
    dp.t.then_count_by_categories(categories=np.arange(num_bins-1), MO=dp.L1Distance[int])
)
noisy_laplace_histogram = dp.binary_search_chain(
    lambda s: histogram >> dp.m.then_laplace(scale=s),
    d_in=1, d_out=epsilon)
sensitive_laplace_counts = noisy_laplace_histogram(df.to_csv(index=False, header=False))
print("Noisy Laplace Counts:", sensitive_laplace_counts[:-1])